In [2]:
%matplotlib inline
from itertools import groupby
from pymongo import MongoClient
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

pdf_pages = PdfPages('MLB 12-07-2017.pdf')
plots_count = 12
plots_per_page = 3
pages_count = int(np.ceil(plots_count / float(plots_per_page)))
grid_size = (plots_per_page, 4)

client = MongoClient("mongodb://localhost:27017/")
db = client["delta_test"]

# MLB tid 70 - 99
teams = db.teams.find({'$and': [{'tid': {'$gte': 70}}, {'tid': {'$lt': 100}}]})
team_names = sorted([team['full'] for team in teams])

def grouper(arg):
    # k= True or False, t = массив серии
    resp = [list(t)for k, t in groupby(arg, lambda x: x > 0)][0]
    return [float(len(resp)), 'g']  if resp[0] > 0 else [float(len(resp)), 'r'] 

mega_list = [[[] for i in range(4)] for r in range(3)] # 4 * 3
game_place, odds_type = ['home + away', 'home', 'away'], ['win/lose', 'handyCap', 'Total', 'iTotal']
short_names = ['Arizona', 'Atlanta', 'Baltimore', 'Boston', 'Ch. Cubs', 'Ch. White Sox', 'Cincinnati',
               'Cleveland', 'Colorado', 'Detroit', 'Houston', 'Kansas City', 'LA Angels', 'LA Dodgers',
               'Miami', 'Milwaukee', 'Minnesota', 'NY Mets', 'NY Yankees', 'Oakland', 'Philadelphia', 
               'Pittsburgh', 'San Diego', 'San Francisco', 'Seattle', 'St.Louis', 'Tampa Bay', 'Texas', 
               'Toronto', 'Washington']

for name in team_names:
    match_list = db.matches.find({'league': 'mlb', 'season': '2015', 'seas_type': 'season',
                                  '$or': [{'home.team': name}, {'away.team': name}]})\
                                  .sort([('date.iso', -1)])
    arry = [[[] for i in range(4)] for r in range(3)] # 4 * 3
    for match in match_list:
        
        """   get delta   """
        delta = match["home"]["ftot"]["delta"] if match["home"]["team"] == name else match["away"]["ftot"]["delta"]
        [arry[0][n].append(delta[n]) for n in range(4)]
        [arry[1][n].append(match["home"]["ftot"]["delta"][n]) for n in range(4) if match["home"]["team"] == name]
        [arry[2][n].append(match["away"]["ftot"]["delta"][n]) for n in range(4) if match["away"]["team"] == name]
    
    # ['Atlanta Braves', [-3.0, 'r']],
    [[mega_list[r][i].append([name, grouper(arry[r][i])]) for i in range(4)] for r in range(3)]

for i in range(4):
    for r in range(3):
        data = [r[1][0] for r in mega_list[r][i]]
        clrs = [r[1][1] for r in mega_list[r][i]]
        dlina = range(len(data))

        fig = plt.figure(figsize=(11.69, 5), dpi=100)
        plt.xticks(dlina, short_names, ha='left', rotation='vertical', fontsize=12)
        ax = plt.axes()
        ax.set_title('Place: {}, type: {}'.format(game_place[r], odds_type[i]))
        bars = ax.bar(dlina, data, color=clrs)
        
        for rect in bars:
            height = rect.get_height() + 0.1
            length = rect.get_x() + rect.get_width() / 2.
            text = int(rect.get_height())
            ax.text(length, height, text, ha='center', va='bottom')
        
        pdf_pages.savefig(fig)

pdf_pages.close()
print('All done')

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it